In [5]:
pip install faiss-cpu langchain-google-genai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 38.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
langchain 0.3.2

In [6]:
import os
import pandas as pd
import sqlite3
import json
import re
from sentence_transformers import SentenceTransformer
import faiss
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI



In [7]:
# Load API key
api_key = userdata.get("GOOGLE_API_KEY") or os.environ.get("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("❌ Gemini API key not found. Add it in Colab Secrets as 'GOOGLE_API_KEY'.")
os.environ["GOOGLE_API_KEY"] = api_key
print("✅ API Key Loaded")

✅ API Key Loaded


In [8]:
# Sales Table
sales = pd.DataFrame({
    "order_id": list(range(1,11)),
    "customer_id": [1,2,1,3,4,2,1,4,3,1],
    "product_id": [101,102,103,101,104,102,101,104,103,101],
    "quantity": [10,5,7,8,6,9,4,7,3,11],
    "price": [100,120,90,110,95,130,80,105,115,98],
    "date": pd.date_range(start="2025-10-01", periods=10).astype(str)
})

# Customers Table
customers = pd.DataFrame({
    "customer_id": [1,2,3,4],
    "customer_name": ["Alice","Bob","Charlie","David"],
    "city": ["Delhi","Mumbai","Kolkata","Chennai"]
})

# Products Table
products = pd.DataFrame({
    "product_id": [101,102,103,104],
    "product_name": ["Laptop","Mouse","Keyboard","Monitor"],
    "category": ["Electronics","Accessories","Accessories","Electronics"]
})

# Regions Table
regions = pd.DataFrame({
    "city": ["Delhi","Mumbai","Kolkata","Chennai"],
    "region": ["North","West","East","South"]
})

# Save CSVs
sales.to_csv("sales.csv", index=False)
customers.to_csv("customers.csv", index=False)
products.to_csv("products.csv", index=False)
regions.to_csv("regions.csv", index=False)
print("✅ Sample CSVs Created")

✅ Sample CSVs Created


In [9]:
sqlite_path = "data_store.db"
conn = sqlite3.connect(sqlite_path)

sales.to_sql("sales", conn, if_exists="replace", index=False)
customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
regions.to_sql("regions", conn, if_exists="replace", index=False)

conn.close()
print(f"✅ Stored CSVs in SQLite: {sqlite_path}")

✅ Stored CSVs in SQLite: data_store.db


In [10]:
conn = sqlite3.connect(sqlite_path)
cursor = conn.cursor()

# Get all tables dynamically
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

metadata = {}
for table in tables:
    cursor.execute(f"PRAGMA table_info({table});")
    schema_info = cursor.fetchall()  # (cid, name, type, notnull, dflt_value, pk)
    schema_df = pd.DataFrame(schema_info, columns=['cid','column_name','column_type','notnull','dflt_value','pk'])

    sample_rows = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5", conn)
    schema_summary = "\n".join([f"{r['column_name']} ({r['column_type']})" for r in schema_df.to_dict('records')])

    metadata[table] = {
        "table": table,
        "schema_df": schema_df,
        "sample_rows": sample_rows,
        "schema_summary": schema_summary
    }

conn.close()
print("✅ Metadata Built Dynamically for Tables:", list(metadata.keys()))

✅ Metadata Built Dynamically for Tables: ['sales', 'customers', 'products', 'regions']


In [11]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

docs = []
table_names = []
for table, meta in metadata.items():
    doc = f"table: {table}\nschema:\n{meta['schema_summary']}\nsample_rows:\n{meta['sample_rows'].to_csv(index=False)}"
    docs.append(doc)
    table_names.append(table)

# Create embeddings
embs = embed_model.encode(docs, convert_to_numpy=True)

# FAISS index
index = faiss.IndexFlatL2(embs.shape[1])
index.add(embs)

# Optional: save embeddings
faiss.write_index(index, 'schema_index.faiss')
with open('schema_docs.json', 'w') as f:
    json.dump(docs, f)

print("✅ Embeddings + FAISS Built")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embeddings + FAISS Built


In [12]:
user_query = "Show average sales price by city including region info"

# Embed the query
q_emb = embed_model.encode([user_query], convert_to_numpy=True)

# Search top 3 relevant tables
D, I = index.search(q_emb, 3)
retrieved_docs = [docs[int(idx)] for idx in I[0]]

print("✅ Retrieved Relevant Docs:")
for doc in retrieved_docs:
    print(doc[:300], "...")  # preview first 300 chars

✅ Retrieved Relevant Docs:
table: regions
schema:
city (TEXT)
region (TEXT)
sample_rows:
city,region
Delhi,North
Mumbai,West
Kolkata,East
Chennai,South
 ...
table: sales
schema:
order_id (INTEGER)
customer_id (INTEGER)
product_id (INTEGER)
quantity (INTEGER)
price (INTEGER)
date (TEXT)
sample_rows:
order_id,customer_id,product_id,quantity,price,date
1,1,101,10,100,2025-10-01
2,2,102,5,120,2025-10-02
3,1,103,7,90,2025-10-03
4,3,101,8,110,2025-10-04
5,4,1 ...
table: customers
schema:
customer_id (INTEGER)
customer_name (TEXT)
city (TEXT)
sample_rows:
customer_id,customer_name,city
1,Alice,Delhi
2,Bob,Mumbai
3,Charlie,Kolkata
4,David,Chennai
 ...


In [13]:
schema_block = "\n\n---\n".join(retrieved_docs)
prompt = (
    f"You are a SQL generation assistant.\n"
    f"Tables: {', '.join(table_names)}\n\n"
    f"Context (schemas & samples):\n{schema_block}\n\n"
    f"User question: \"{user_query}\"\n\n"
    "Generate:\n"
    "1) A valid, safe SQL SELECT query for SQLite (use exact table names)\n"
    "2) A one-line explanation.\n"
    "Respond exactly with labeled blocks:\nSQL Query:\n<SQL>\n\nExplanation:\n<one-line>"
)
print("✅ Prompt Built")

✅ Prompt Built


In [14]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=api_key)
resp = llm.invoke(prompt)
llm_output = resp.content
print("✅ Gemini Response Received")
print(llm_output)

✅ Gemini Response Received
SQL Query:
```sqlite
SELECT
  c.city,
  r.region,
  AVG(s.price) AS average_sales_price
FROM sales AS s
JOIN customers AS c
  ON s.customer_id = c.customer_id
JOIN regions AS r
  ON c.city = r.city
GROUP BY
  c.city,
  r.region;
```

Explanation:
This query calculates the average sales price for each city, including its region, by joining sales, customers, and regions tables and grouping the results by city and region.


In [15]:
sql = None
explanation = None
text = llm_output

if text:
    if 'SQL Query:' in text:
        after = text.split('SQL Query:', 1)[1]
        if 'Explanation:' in after:
            sql_part, expl_part = after.split('Explanation:', 1)
            sql = sql_part.strip().strip('`').replace('sqlite','').strip()
            explanation = expl_part.strip()
        else:
            sql = after.strip().splitlines()[0].strip().strip('`').replace('sqlite','').strip()
            explanation = text.replace(sql, '').strip()
    else:
        m = re.search(r"(SELECT[\s\S]+?;?)", text, flags=re.IGNORECASE)
        if m:
            sql = m.group(1).strip().strip('`').replace('sqlite','').strip()
            explanation = text.replace(sql, '').strip()
        else:
            explanation = text.strip()

parsed_sql = {"sql": sql, "explanation": explanation}
print("✅ Parsed SQL")
print(parsed_sql)

✅ Parsed SQL
{'sql': 'SELECT\n  c.city,\n  r.region,\n  AVG(s.price) AS average_sales_price\nFROM sales AS s\nJOIN customers AS c\n  ON s.customer_id = c.customer_id\nJOIN regions AS r\n  ON c.city = r.city\nGROUP BY\n  c.city,\n  r.region;', 'explanation': 'This query calculates the average sales price for each city, including its region, by joining sales, customers, and regions tables and grouping the results by city and region.'}


In [16]:
conn = sqlite3.connect(sqlite_path)

if parsed_sql['sql'] and parsed_sql['sql'].lower().startswith('select'):
    df_result = pd.read_sql_query(parsed_sql['sql'], conn)
else:
    df_result = None

conn.close()
print("✅ Query Result:")
print(df_result)

✅ Query Result:
      city region  average_sales_price
0  Chennai  South                100.0
1    Delhi  North                 92.0
2  Kolkata   East                112.5
3   Mumbai   West                125.0


In [17]:
import os
if os.path.exists(sqlite_path):
    os.remove(sqlite_path)
    print("🧹 Removed", sqlite_path)

🧹 Removed data_store.db
